# (1) 데이터 불러오기

In [1]:
import json
import pandas as pd
import numpy as np
with open('D:/플레이리스트_24/genre_gn_all.json', 'r', encoding='utf-8') as f:

    json_data_genre = json.load(f)
pd_genre = pd.DataFrame(list(json_data_genre.keys()),list(json_data_genre.values())).reset_index(drop=False)
pd_genre.columns=['genre', 'code']


with open('D:/플레이리스트_24/song_meta.json', 'r', encoding='utf-8') as f:

    json_data_song = json.load(f)
pd_song=pd.DataFrame(json_data_song)

In [2]:
with open('D:/플레이리스트_24/train.json', 'r', encoding='utf-8') as f:

    json_data_train = json.load(f)
pd_train=pd.DataFrame(json_data_train)

with open('D:/플레이리스트_24/test.json', 'r', encoding='utf-8') as f:

    json_data_test = json.load(f)
pd_test=pd.DataFrame(json_data_test)    

with open('D:/플레이리스트_24/val.json', 'r', encoding='utf-8') as f:

    json_data_val = json.load(f)
pd_val=pd.DataFrame(json_data_val)

tag_data=pd.concat([pd_train,pd_test, pd_val]).reset_index(drop=True)
tag_data=tag_data.set_index(['id'])

# (2) 태그 데이터 제거

In [3]:
# 결측값 제거
pd_data_na=tag_data[(tag_data['songs'].apply(len)!=0) & (tag_data['tags'].apply(len)!=0)]
pd_data_na=pd_data_na.reset_index(drop=False)

In [4]:
from collections import Counter
tags_series = pd_data_na['tags'].explode()
c = tags_series.tolist()

tag_counts = Counter(c)
df_tag_counts = pd.DataFrame(tag_counts.items(), columns=['tag', 'count'])
df_tag_counts = df_tag_counts.sort_values(by='count', ascending=False)

In [5]:
# 특수문자 제거
import re
def remove_special_characters(text):
    return re.sub(r'[^\w\s]|_', '', text)

df_tag_counts['tag'] = df_tag_counts['tag'].apply(remove_special_characters)

In [6]:
# 장르 제거
words_to_remove = pd_genre['genre'].str.lower()

def remove_words(text, words):
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in words) + r')\b'
    return re.sub(pattern, '', text)

df_tag_counts['text_column'] = df_tag_counts['tag'].str.lower().apply(lambda x: remove_words(x, words_to_remove))

In [7]:
# 숫자 제거
def remove_numbers(cell):
    if re.search(r'\d', str(cell)):
        return ''
    else:
        return cell

df_tag_counts['num_drop'] = df_tag_counts['tag'].apply(remove_numbers)

In [8]:
# 제거 후 결측값 행 삭제
df_delete=df_tag_counts[(df_tag_counts['text_column'] != "") & (df_tag_counts['num_drop'] != "")][['tag','count']]
df_delete.columns = ['num', 'count']

In [10]:
# 노래, 음악, 차트, 플레이리스트 태그 제거
import pandas as pd
import re

words_to_remove = ['노래', '음악', '차트', '플레이리스트']

def remove_words(text, words):
    pattern = re.compile('|'.join([re.escape(word) for word in words]))
    return pattern.sub('', text)

df_delete['num'] = df_delete['num'].apply(lambda x: remove_words(x, words_to_remove))

data_delete=df_delete[df_delete['num']!=""]

In [12]:
# 한글자 의미있는 태그만 남기고 데이터 삭제
df_delete_1=data_delete[data_delete['num'].apply(len)!=1]

add_data=pd.concat([data_delete[data_delete['num'].apply(len)==1].iloc[0:6],data_delete[data_delete['num'].apply(len)==1].iloc[7:19], data_delete[data_delete['num'].apply(len)==1].iloc[20:25],data_delete[data_delete['num'].apply(len)==1].iloc[31:32]])

df_data=pd.concat([df_delete_1,add_data]).sort_values(by='count',ascending = False)

In [13]:
# 태그 빈도 19 이상만 추출
df_data=pd.concat([df_delete_1,add_data]).sort_values(by='count',ascending = False)
list_tag=df_data[df_data['count']>=19]

# (3) 태그 gpt로 전처리(분위기, 장소, 시간, 날씨, 감정, 상황 태그만 추출)

# (4) 연령대 태그 생성

In [14]:
#gpt로 전처리된 태그 데이터
df_태그_전처리 = pd.read_csv("태그_개수포함_240725.csv", encoding="cp949")

In [15]:
from collections import Counter

tags_series = pd_data_na['tags'].explode()
song_series = pd_data_na['songs'].explode()

tag_song=pd.merge(pd.DataFrame(tags_series).reset_index(drop=False),pd.DataFrame(song_series).reset_index(drop=False),left_on='index', right_on='index', how='outer')

In [16]:
tag_song_list=pd.merge(tag_song, pd_song,left_on='songs', right_on='id', how='left')
년도=pd.concat([tag_song_list[tag_song_list['tags']=="8090"], tag_song_list[tag_song_list['tags']=="7080"], tag_song_list[tag_song_list['tags']=="6070"], tag_song_list[tag_song_list['tags']=="5060"], tag_song_list[tag_song_list['tags']=="3040"], tag_song_list[tag_song_list['tags']=="40대"], tag_song_list[tag_song_list['tags']=="30대"], tag_song_list[tag_song_list['tags']=="20대"], tag_song_list[tag_song_list['tags']=="10대"]])

년도['num']=년도['tags'].replace({'7080':'50대 이상', '8090':'30~40대', '20대':'10~20대', '30대':'30~40대', '10대':'10~20대', '3040':'30~40대', '6070':'50대 이상', '5060':'50대 이상', '40대':'30~40대'})

# 플레이리스트 전처리된 태그만 추출

In [17]:
import pandas as pd
import re
def remove_special_characters(text):
    # 특수문자를 공백으로 대체
    return re.sub(r'[^\w\s]|_', '', text)



tag_song['tags'] = tag_song['tags'].apply(remove_special_characters)

In [18]:
import re
def remove_numbers(cell):
    if re.search(r'\d', str(cell)):
        return ''
    else:
        return cell


tag_song['num'] = tag_song['tags'].apply(remove_numbers)

In [19]:
import re
def remove_numbers(cell):
    if re.search(r'\d', str(cell)):
        return ''
    else:
        return cell


tag_song['num'] = tag_song['tags'].apply(remove_numbers)

In [20]:
df_delete=tag_song[tag_song['num'] != ""]

In [21]:
import pandas as pd
import re

words_to_remove = ['노래', '음악', '차트', '플레이리스트']

def remove_words(text, words):

    pattern = re.compile('|'.join([re.escape(word) for word in words]))

    return pattern.sub('', text)


df_delete['num'] = df_delete['num'].apply(lambda x: remove_words(x, words_to_remove))

C:\Users\ahn\AppData\Local\Temp\ipykernel_31888\2666524126.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_delete['num'] = df_delete['num'].apply(lambda x: remove_words(x, words_to_remove))


In [22]:
data_delete=df_delete[df_delete['num']!=""]
df_delete_1=data_delete[data_delete['num'].apply(len)!=1]

In [23]:
import pandas as pd


search_strings = [
    "밤","팝","락","봄","랩","비","눈","썸","집","잠","흥","펍","책","꿈","별","춤","꽃","술","힘","일","힙","쉼","차","낮"
]

add_data = data_delete[data_delete['num'].isin(search_strings)]

add_data=pd.DataFrame(add_data)

In [24]:
df_data_final=pd.concat([df_delete_1,add_data])

In [25]:
search_list = list(df_태그_전처리['단어'])
filtered_df = df_data_final[df_data_final['num'].isin(search_list)]
filtered_df
# 연령대 태그 결합
년도=년도[['index', 'num','songs']]
filtered_df=filtered_df[['index','num','songs']]
filtered_data =pd.concat([filtered_df, 년도]).reset_index(drop=True)
filtered_df_tag = filtered_data.drop_duplicates(subset=['index', 'num'])
filtered_df_series_1=filtered_df_tag.set_index('index')
filtered_df_series_2=filtered_data.set_index('index')
플레이리스트=pd.concat([filtered_df_series_2['songs'].groupby(filtered_df_series_2.index).apply(set), filtered_df_series_1['num'].groupby(filtered_df_series_1.index).apply(list)], axis=1)
플레이리스트['songs']=플레이리스트['songs'].apply(list)

In [26]:
최종플레이리스트=pd.merge(플레이리스트.reset_index(drop=False), pd_data_na.reset_index(drop=False), left_on='index', right_on='index', how='left')[['index', 'num', 'plylst_title','songs_y','like_cnt','updt_date']]
최종플레이리스트.columns = ['index', 'tags', 'plylst_title','songs','like_cnt','updt_date']
최종플레이리스트.to_csv("최종플레이리스트(1).csv", encoding='utf-8')

# 노래 데이터 -> 플레이리스트에 있는 곡만 추출

In [27]:
from collections import Counter

tags_series = 플레이리스트['num'].explode()
song_series = 플레이리스트['songs'].explode()

플레이리스트_tag_song=pd.merge(pd.DataFrame(tags_series).reset_index(drop=False),pd.DataFrame(song_series).reset_index(drop=False),left_on='index', right_on='index', how='outer')
플레이리스트_tag_song=플레이리스트_tag_song.drop_duplicates(subset=['num', 'songs'])[['num','songs']]

In [28]:
song_list_final1=플레이리스트_tag_song.set_index('songs')
song_list_tag=pd.DataFrame(song_list_final1['num'].groupby(song_list_final1.index).apply(list)).reset_index(drop=False)
song_list_tag.columns=['songs','tags']
최종_songlist=pd.merge(pd_song, song_list_tag, left_on='id', right_on='songs', how='left').dropna()

In [29]:
# 장르 코드를 텍스트로 변환, 출시날짜 -> 출시연도
장르변경=최종_songlist.set_index('id')
장르변경_a=pd.DataFrame(장르변경['song_gn_gnr_basket'].explode())
장르변경_a=장르변경_a.reset_index(drop=False)
장르변경_b=pd.merge(장르변경_a, pd_genre, left_on='song_gn_gnr_basket', right_on='code', how='left').set_index('id')
장르변경_c=장르변경_b['genre'].groupby(장르변경_b.index).apply(list)
장르변경_c=장르변경_c.reset_index(drop=False)
장르변경_최종=pd.merge(최종_songlist.reset_index(drop=False), 장르변경_c, left_on='id', right_on='id', how='left')
장르변경_최종['발매_year']=장르변경_최종['issue_date'].str[0:4]
장르변경_최종['발매_year'] = 장르변경_최종['발매_year'].replace('0000', "")
장르변경_최종[['song_gn_dtl_gnr_basket','issue_date','album_name','album_id','artist_id_basket','song_name','song_gn_gnr_basket','artist_name_basket','id','tags','genre','발매_year']]
장르변경_최종.to_csv("최종노래목록(1).csv", encoding="cp949")

# 성능평가 데이터

### (1) 플레이리스트 데이터

In [30]:
with open('train.json', 'r', encoding='utf-8') as f:
    json_data_train = json.load(f)
pd_train_1=pd.DataFrame(json_data_train)

In [31]:
pd_train_1['tags'] = pd_train_1['tags'].apply(lambda x: str(x))
pd_train_1['songs'] = pd_train_1['songs'].apply(lambda x: str(x))

In [32]:
pd_data_na_index=pd_data_na.reset_index(drop=False)
pd_data_na_index['tags']=pd_data_na_index['tags'].apply(lambda x: str(x))
pd_data_na_index['plylst_title']=pd_data_na_index['plylst_title'].apply(lambda x: str(x))
pd_data_na_index['songs']=pd_data_na_index['songs'].apply(lambda x: str(x))

In [33]:
pd_train_1['tags']=pd_train_1['tags'].apply(lambda x: str(x))
pd_train_1['plylst_title']=pd_train_1['plylst_title'].apply(lambda x: str(x))
pd_train_1['songs']=pd_train_1['songs'].apply(lambda x: str(x))

In [34]:
결합=pd.merge(pd_train_1,pd_data_na_index, left_on=['tags', 'plylst_title', 'songs'], right_on=['tags', 'plylst_title', 'songs'], how='left')

In [35]:
결합_1=결합[['index', 'tags','plylst_title','like_cnt_x','updt_date_x']]

In [36]:
결합데이터_data=pd.read_csv("최종플레이리스트(1).csv",encoding='utf-8')

In [37]:
플레이리스트_전체 = pd.merge(결합_1, 결합데이터_data, left_on='index', right_on='index', how='left')

In [38]:
플레이리스트_=플레이리스트_전체[['index', 'tags_y','plylst_title_y','songs', 'like_cnt','updt_date']].dropna(subset=['tags_y'])

In [39]:
플레이리스트_.columns = ['index', 'tag', 'plyst_title', 'songs', 'like_cnt', 'updt_date']

In [40]:
플레이리스트_=플레이리스트_.sort_values(by='index').reset_index(drop=True)

In [41]:
플레이리스트_.to_json('플레이리스트_train_최종(1).json')

### (2) 노래목록 데이터

In [42]:
import ast
플레이리스트_['tag']=플레이리스트_['tag'].apply(ast.literal_eval)
플레이리스트_['songs']=플레이리스트_['songs'].apply(ast.literal_eval)

In [43]:
from collections import Counter

tags_series = 플레이리스트_['tag'].explode()
song_series = 플레이리스트_['songs'].explode()

플레이리스트_tag_song=pd.merge(pd.DataFrame(tags_series).reset_index(drop=False),pd.DataFrame(song_series).reset_index(drop=False),left_on='index', right_on='index', how='outer')
플레이리스트_tag_song=플레이리스트_tag_song.drop_duplicates(subset=['tag', 'songs'])[['tag','songs']]

In [44]:
song_list_final1=플레이리스트_tag_song.set_index('songs')
song_list_tag=pd.DataFrame(song_list_final1['tag'].groupby(song_list_final1.index).apply(list)).reset_index(drop=False)

In [45]:
최종_songlist=pd.merge(pd_song, song_list_tag, left_on='id', right_on='songs', how='left').dropna()

In [46]:
장르변경=최종_songlist.set_index('id')
장르변경_a=pd.DataFrame(장르변경['song_gn_gnr_basket'].explode())
장르변경_a=장르변경_a.reset_index(drop=False)
장르변경_b=pd.merge(장르변경_a, pd_genre, left_on='song_gn_gnr_basket', right_on='code', how='left').set_index('id')
장르변경_c=장르변경_b['genre'].groupby(장르변경_b.index).apply(list)
장르변경_c=장르변경_c.reset_index(drop=False)
장르변경_최종=pd.merge(최종_songlist.reset_index(drop=False), 장르변경_c, left_on='id', right_on='id', how='left')
장르변경_최종['발매_year']=장르변경_최종['issue_date'].str[0:4]
장르변경_최종['발매_year'] = 장르변경_최종['발매_year'].replace('0000', "")
장르변경_최종[['song_gn_dtl_gnr_basket','issue_date','album_name','album_id','artist_id_basket','song_name','song_gn_gnr_basket','artist_name_basket','id','tag','genre','발매_year']]
장르변경_최종.to_json('노래목록_train_최종(1).json')